__качаем объявления о продаже квартир с циан.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
import sys
import re
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import random

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [3]:
raw_data_path = 'data/raw/flat/cian' # каталог с файлами объявлений

In [4]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'
# base_url, req_param = 'https://sevastopol.cian.ru/kupit-kvartiru/', ''
place = 'sevastopol'
base_url, req_param ='https://www.cian.ru/cat.php', 'deal_type=sale&offer_type=flat&region=184723'

In [5]:
%%time

# from lib.downloader import DownloaderSimple
# from lib.parser import AdsListParser
from lib.downloader import DownloaderSeleniumFirefox
from lib.cian import CianParser


df,html = CianParser(
        base_url = base_url,
        driver=DownloaderSeleniumFirefox(profile_path),
        # driver=DownloaderSimple(),
    ).load(req_param=req_param, keep_html=True) # , npage_start=53) # , page_limit=3 ) #

print(len(df))

[INFO    ] 2022-11-04 17:05:50 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-04 17:05:50 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-04 17:05:52 | AdsListParser: downloader init
[INFO    ] 2022-11-04 17:05:52 | CianParser: downloader init
[INFO    ] 2022-11-04 17:05:52 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-04 17:06:09 | AdsListParser: read page 1
[INFO    ] 2022-11-04 17:06:19 | AdsListParser: read page 2
[INFO    ] 2022-11-04 17:06:26 | AdsListParser: read page 3
[INFO    ] 2022-11-04 17:06:41 | AdsListParser: read page 4
[INFO    ] 2022-11-04 17:06:54 | AdsListParser: read page 5
[INFO    ] 2022-11-04 17:06:58 | AdsListParser: read page 6
[INFO    ] 2022-11-04 17:07:08 | AdsListParser: read page 7
[INFO    ] 2022-11-04 17:07:24 | AdsListParser: read page 8
[INFO    ] 2022-11-04 17:07:36 | AdsListParser: read page 9
[INFO    ] 2022-11-04 17:07:50 | AdsListParser: read page 10
[INFO    ] 2022-11-04 17:07:55 | AdsLis

In [6]:
df.sample(3)

,OfferTitle,OfferSubtitle,Deadline,MainPrice,PriceInfo,GeoLabel,TimeLabel,LinkArea,Description,page,ts
988,"1-комн. кв., 50 м², 1/5 этаж",,,6 900 000 ₽,138 000 ₽/м²,"[Севастополь, р-н Нахимовский, мкр. Горпищенко, улица Горпищенко, 94]","[неделю назад, 22 окт, 12:29]",https://sevastopol.cian.ru/sale/flat/272730094/,"Однокомнатная квартира 50 м2, на 1/5 эт. дома с просторной кухней 13,5 м2. Расположена в г. Севастополь, Нахимовский район, ул. Горпищенко, д. 94.\nПозвоните нам, чтобы узнать все детали! \nТехнические характеристики: \n \n-Адрес: г. Севастополь, Нахимовский район, ул. Горпищенко, д. 94\n-Год постройки дома: 1986\n-Этаж/этажность: 1/5\n-Количество комнат: 1\n-Комнаты: изолированные\n-Общая площадь: 50 м2\n-Жилая площадь: 19,8 м2\n-Комната: 18,9 м2\n-Площадь кухни: 13,5 м2\n-Высота потолка: 2,5 м \n-Санузел: совмещенный \n-Водоснабжение: центральное \n-Горячее водоснабжение: бойлер \n-Отопление: центральное \nДополнительная информация: \nКвартира расположена на первом очень высоком этаже пятиэтажного дома. В квартире сделан качественный косметический ремонт. Вся мебель остается новым хозяевам!\nСтены по периметру лоджии утеплены, что гарантирует комфорт и тепло в квартире. За счет объединения кухни с лоджией (13,5м2) значительно увеличилась столовая зона, что добавляет комфорта жильцам и гостям дома. Двухуровневые гипсокартонные потолки с несколькими режимами подсветки, обеспечивают дополнительный уют. Просторная прихожая позволяет оборудовать вместительную зону гардеробной и зону хранения. \nСовмещенный санузел с ванной общей площадью 5,5 м2. и установленной стиральной машинкой.\n Особое внимание дополнительному вместительному подсобному помещению, помогающему решить хозяйственно-бытовые проблемы. \nПозвоните нам, чтобы назначить просмотр! \nТолько наличный расчет. \nДокументы РФ проверены юристами и готовы к сделке. \nПрофессиональное сопровождение до получения права собственности.\nДобавьте предложение в закладки, чтобы не потерять!",36,2022-11-04 17:14:54.237934
138,"2-комн. кв., 60,6 м², 5/8 этаж",Секция 1 • Сдача корпуса 1 кв. 2024,сдача ГК: 1 кв. 2024 года,8 181 000 ₽,135 000 ₽/м²,"[Севастополь, р-н Нахимовский, проспект Победы, 29Б]","[11 часов назад, сегодня, 05:31]",https://sevastopol.cian.ru/sale/flat/272040036/,"Продаётся 2-комнатная квартира в строящемся доме (Корабельный), срок сдачи: I-кв. 2024, общей площадью 60.6 кв.м., на 5 этаже. ЖК Корабельный - новый объект компании ИнтерСтрой, расположенный в самом сердце Нахимовского района. Этот комфортный жилой комплекс станет идеальным выбором для тех, кто ценит спокойствие и уют, но в то же время хочет жить в непосредственной близости к центру города. \nУдобное расположение \nЖК строится на проспекте Победы - в районе с хорошо развитой инфраструктурой. \nПоблизости находятся излюбленные места отдыха горожан - знаменитый парк Малахов Курган, недавно отреставрированный мемориальный комплекс Сапун-Гора, эко-парк Лукоморье с детскими аттракционами и небольшим зоопарком. \nВ шаговой доступности несколько школ и детских садов, продуктовый рынок, магазины и супермаркеты. Остановка общественного транспорта в 2х минутах ходьбы. \nДостоинства спального района удачно дополняют удобные транспортные развязки и подъездные пути: до автобусного и железнодорожного вокзалов можно доехать за 10 минут, до центра Севастополя - всего за 15-20 минут. \nСтандарты строительства \nВ ЖК Корабельный воплощены современные строительные решения и технологии. \nОсобого внимания достойны уютные дизайнерские холлы. \nВсе балконы и лоджии в доме застеклены. Качественные материалы обеспечивают высокую тепло- и звукоизоляцию. \nДом стоит в отдалении от соседних построек, поэтому квартиры наполнены светом и воздухом, а из окон открывается привлекательная панорама района. \nДля сохранения полезного пространства в квартирах есть возможность приобрести отдельные кладовые, а для безопасной парковки личного авто - крытое паркоместо. \nРайон с богатой историей \nКорабельная сторона - место, где заро

In [7]:
print(len(df))
df = pd.DataFrame(df).dropna().drop_duplicates(['LinkArea','Description',]).reset_index(drop=True)
print(len(df))

1495
1444


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
# df.to_excel(f'{raw_data_path}/cian_{ts}_raw.xlsx',index=False)
df.to_pickle(f'{raw_data_path}/cian_{ts}_raw.pkl')

----

In [10]:
# df = pd.read_excel('data/raw/cian/cian_2022-09-08_13-49_raw.xlsx')

In [11]:
# df

In [12]:
# df['GeoLabel'].apply( lambda s : len(s) ) #', '.join(s) )

# s = ['Севастополь', 'р-н Гагаринский', 'Казачья Бухта мкр', 'Скифия ЖК']
# ', '.join(s)

In [13]:
# from lib.cian import CianDataCleaner

# CianDataCleaner().transform(df)

----

In [14]:
# with open('tmp/cian.html','wt') as f: f.write(html[0])